In [ ]:
## Bibliothèques générales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from datetime import datetime

#print("bibliotheques importés")

NumPy: 2.0.2
Pandas: 2.2.2


## Importations des données

In [ ]:
from google.colab import files

# Téléversement du fichier Excel
print("➡️ Téléversez le fichier Excel...")
uploaded_files = files.upload()

# Vérifier si un fichier a été téléversé
if not uploaded_files:
    raise Exception("❌ Aucun fichier téléversé.")

# Récupérer le nom du fichier téléversé
uploaded_filename = list(uploaded_files.keys())[0]

# Chargement du fichier Excel
df = pd.read_excel(uploaded_filename)


➡️ Téléversez le fichier Excel...


Saving data_final_entre_2016_2024.xlsx to data_final_entre_2016_2024.xlsx


In [ ]:
df.columns

Index(['PMID', 'Pays', 'Date', 'Affiliation', 'Authors_Countries',
       'Detected_Viruses', 'Detected_Genus', 'Detected_Families',
       'clé_composite'],
      dtype='object')

In [ ]:
df.head()

,PMID,Pays,Date,Affiliation,Authors_Countries,Detected_Viruses,Detected_Genus,Detected_Families,clé_composite
0,26854905,Algeria,2016-02-09,"micalis institute, inra, agroparistech, univer...",Algérie; Brésil; France,['Human papillomavirus'],['Papillomavirus'],['Papillomaviridae'],26854905_Algeria
1,26856380,Algeria,2016-02-10,"laboratoire des hépatites virales, institut pa...",Algérie; France,['Hepatitis C virus'],['Hepacivirus'],['Flaviviridae'],26856380_Algeria
2,27085765,Algeria,2016-04-18,"université de tlemcen, laboratoire de microbio...",Algérie; France; Maroc,['Human papillomavirus'],['Papillomavirus'],['Papillomaviridae'],27085765_Algeria
3,27330333,Algeria,2016-06-23,laboratory of applied molecular biology and im...,Algérie,['Epstein-Barr virus'],['Lymphocryptovirus'],['Herpesviridae'],27330333_Algeria
4,27534938,Algeria,2016-08-19,laboratoire ecologie des systèmes terrestres e...,Algérie; Belgique; France,"['Rift valley fever virus, West Nile virus']","['Flavivirus, Phlebovirus']","['Flaviviridae, Phenuiviridae']",27534938_Algeria


In [ ]:
## Conversion extraction de l'année
data_original=df.copy()
# Conversion de la colonne 'Date' en datetime
data_original["Date"] = pd.to_datetime(data_original["Date"], format="%Y/%m/%d")
data_original["year"]=data_original["Date"].dt.year


In [ ]:
## Nettoyage  de Données
data_original['Pays'] =data_original['Pays'].replace('South Africa1', 'South Africa')

corrections_pays = {
    "Guine Bissau": "Guinea-Bissau",
    "Ivory Coast": "Côte d'Ivoire",
    'S╞o Tomé and Príncipe': 'Sao Tome and Principe'
}

## Remplacement dans la colonne "Pays"
data_original["Pays"] = data_original["Pays"].replace(corrections_pays)

## selectons des années
data_original = data_original[(data_original["year"] < 2025) & (data_original["year"] > 1940)]


In [ ]:
## creations des colonne l'avant et de l'apres pandemie
data_original["pandemie"] = np.where(data_original["year"] < 2020, "Before pandemic", "After pandemic")

In [ ]:

data_original.head(3)

,PMID,Pays,Date,Affiliation,Authors_Countries,Detected_Viruses,Detected_Genus,Detected_Families,clé_composite,year,pandemie
0,26854905,Algeria,2016-02-09,"micalis institute, inra, agroparistech, univer...",Algérie; Brésil; France,['Human papillomavirus'],['Papillomavirus'],['Papillomaviridae'],26854905_Algeria,2016,Before pandemic
1,26856380,Algeria,2016-02-10,"laboratoire des hépatites virales, institut pa...",Algérie; France,['Hepatitis C virus'],['Hepacivirus'],['Flaviviridae'],26856380_Algeria,2016,Before pandemic
2,27085765,Algeria,2016-04-18,"université de tlemcen, laboratoire de microbio...",Algérie; France; Maroc,['Human papillomavirus'],['Papillomavirus'],['Papillomaviridae'],27085765_Algeria,2016,Before pandemic


In [ ]:
 ## Compter les occurrences des articles
article_par_pays = data_original.groupby(["Pays", 'pandemie']).size().reset_index(name='number_of_articles')
article_par_pays = article_par_pays.sort_values(by='number_of_articles', ascending=False)

article_par_pays.sample(3)

,Pays,pandemie,number_of_articles
88,South Africa,Before pandemic,2309
13,Cameroon,After pandemic,687
69,Namibia,After pandemic,115


Évolution du nombre d’articles dans les pays africains : une analyse avant et après la COVID-19

In [ ]:
import plotly.express as px

# Graph sur les  nombres de  publications par pays
fig = px.bar(article_par_pays,
             y='number_of_articles',
             x='Pays',
             color='pandemie',

             title='<b>Nombre d\'articles par pays avant et après la pandémie</b>',
             labels={
                 'number_of_articles': '<b>Nombre d\'articles</b>',
                 'Pays': '<b>Pays</b>',
                 'pandemie': '<b>Période pandémique</b>'
             })


fig.update_layout(
    title_font_size=18,
    title_x=0.5,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    legend_title_font_size=14,
    hoverlabel_font_size=12,
    plot_bgcolor='white'
)

fig.update_xaxes(tickangle=-45, tickfont=dict(size=12))
fig.update_yaxes(gridcolor='lightgrey', gridwidth=0.5)

fig.show()

In [ ]:
# Barplot pour les articles par année

article_par_year = data_original["year"].value_counts().reset_index()

# Renommer correctement les colonnes
article_par_year.columns = ['year', 'numbers of article']
article_par_year = article_par_year.sort_values('year')

article_par_year.sample(5)


,year,numbers of article
2,2020,5167
0,2021,6876
5,2017,2916
3,2024,4487
4,2023,4102


## Évolution du nombre d’articles dans les pays africains (2016–2024) : une analyse avant et après la COVID-19

In [ ]:
import plotly.express as px
import numpy as np

article_par_year["pandemie"] = np.where(article_par_year["year"] < 2020,
                                        "Before pandemic",
                                        "After pandemic")

fig = px.line(
    article_par_year,
    x="year",
    y="numbers of article",
    color="pandemie",
    markers=True,
    color_discrete_map={
        "Before pandemic": "#66c2a5",
        "After pandemic": "#fc8d62"
    },
    title="Evolution of Publications Before and After the Pandemic"
)


fig.update_layout(
    title_font_size=18,
    title_x=0.5,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    legend_title_font_size=16,
    hoverlabel_font_size=12  #,
    #plot_bgcolor='#15b7ea'
)

fig.update_xaxes(tickangle=0, tickfont=dict(size=12))
fig.update_yaxes(gridcolor='lightgrey', gridwidth=0.7)

# Ligne verticale pour 2020
fig.add_vline(x=2020, line_dash="dash", line_color="red", annotation_text="Pandemic start")

fig.show()


In [ ]:
%pip install pycountry
%pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.1 MB/s eta 0:00:00


In [ ]:
## Standardisation des noms de pays pour leur intégration cartographique, accompagnée de l’ajout des codes ISO3

In [ ]:
import pycountry

### Obtenir la liste des noms officiels des pays en anglais
pycountry_names = set([country.name for country in pycountry.countries])

### Ajouter les noms courants (common_name), noms officiels (official_name), etc.
for country in pycountry.countries:
    if hasattr(country, 'official_name'):
        pycountry_names.add(country.official_name)
    if hasattr(country, 'common_name'):
        pycountry_names.add(country.common_name)

### Vérifier les pays non reconnus dans ta colonne "Pays"
pays_non_reconnus = set()

for nom in article_par_pays['Pays'].unique():
    if nom not in pycountry_names:
        pays_non_reconnus.add(nom)

### Afficher les noms de pays non reconnus

if len(pays_non_reconnus) > 0:
    print("Pays non reconnus par pycountry :")
    for nom in sorted(pays_non_reconnus):
        print("\t-", nom)
else:
    print("Tous les pays sont reconnus par pycountry")

Tous les pays sont reconnus par pycountry


In [ ]:
article_by_year_contry=article_par_pays

article_by_year_contry.head(3)

,Pays,pandemie,number_of_articles
27,Egypt,After pandemic,4263
87,South Africa,After pandemic,3704
88,South Africa,Before pandemic,2309


In [ ]:
article_by_country=article_by_year_contry


import pycountry

def country_name_to_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

# Ajouter la colonne avec le code ISO3 du pays
article_by_country['country'] = article_by_country['Pays'].apply(country_name_to_iso3)

# Afficher les lignes où la conversion en code ISO3 a échoué
lignes_avec_na = article_by_country[article_by_country['country'].isna()]

print(lignes_avec_na)




Empty DataFrame
Columns: [Pays, pandemie, number_of_articles, country]
Index: []


In [ ]:
#article_by_country.loc[article_by_country["Pays"] == "Congo", "country"] = "COD"

In [ ]:
#article_by_country[article_by_country["country"]=="COD"]

## Carte  de  publication par année et  pays

In [ ]:
article_by_year_contry = data_original.groupby(["Pays", "year"]).size().reset_index(name="numbers of article").sort_values("year")
article_by_year_contry.loc[article_by_year_contry["Pays"] == "Congo", "country"] = "COD"

def country_name_to_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

# Ajouter la colonne avec le code ISO3 du pays
article_by_year_contry['country'] = article_by_year_contry['Pays'].apply(country_name_to_iso3)

In [ ]:
article_by_year_contry.sample(3)

,Pays,year,numbers of article,country
171,Ghana,2023,164,GHA
274,Morocco,2018,41,MAR
132,Eswatini,2020,16,SWZ


In [ ]:
import plotly.express as px
article_by_year_contry = data_original.groupby(["Pays", "year"]).size().reset_index(name="numbers of articles").sort_values("year")
article_by_year_contry.loc[article_by_year_contry["Pays"] == "Congo", "country"] = "COD"

def country_name_to_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

# Ajouter la colonne avec le code ISO3 du pays
article_by_year_contry['country'] = article_by_year_contry['Pays'].apply(country_name_to_iso3)

# Afficher les lignes où la conversion en code ISO3 a échoué
lignes_avec_na = article_by_year_contry[article_by_year_contry['country'].isna()]

print(lignes_avec_na)


fig = px.choropleth(
    article_by_year_contry,
    locations="country",            # colonne des codes ISO alpha-3
    color="numbers of articles",     # variable à colorer
    hover_name="Pays",              # nom affiché au survol
    color_continuous_scale=px.colors.sequential.Plasma,
    scope="africa",                 # limitations à l'Afrique si tu veux
    animation_frame="year",
    labels={"numbers of articles": "Nombre d'articles"},
    title="Number of articles per African country per year",
    range_color=[20,80]
)

fig.show()


Empty DataFrame
Columns: [Pays, year, numbers of articles, country]
Index: []


In [ ]:
import plotly.express as px
article_by_year_contry = data_original.groupby(["Pays", "year"]).size().reset_index(name="numbers of article").sort_values("year")
article_by_year_contry.loc[article_by_year_contry["Pays"] == "Congo", "country"] = "COD"

def country_name_to_iso3(name):
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

# Ajouter la colonne avec le code ISO3 du pays
article_by_year_contry['country'] = article_by_year_contry['Pays'].apply(country_name_to_iso3)

# Afficher les lignes où la conversion en code ISO3 a échoué
lignes_avec_na = article_by_year_contry[article_by_year_contry['country'].isna()]

print(lignes_avec_na)


fig = px.choropleth(
    article_by_year_contry,
    locations="country",            # colonne des codes ISO alpha-3
    color="numbers of article",     # variable à colorer
    hover_name="Pays",              # nom affiché au survol
    color_continuous_scale=px.colors.sequential.Plasma,
    scope="africa",                 # limiter à l'Afrique si tu veux
    animation_frame="year",
    labels={"numbers of article": "Nombre d'articles"},
    title="Number of articles per African country per year",
    range_color=[20,80]
)

fig.show()


Empty DataFrame
Columns: [Pays, year, numbers of article, country]
Index: []
